### 모듈 선언과 기초 설정

In [37]:
from random import shuffle
import pygame

pygame.init()

size = [700, 600] # 윈도우 크기
screen = pygame.display.set_mode(size) # 디스플레이
clock = pygame.time.Clock() # 시간 관리 객체
font = pygame.font.Font("fonts/NanumSquareR.ttf", 16) # 폰트

pygame.display.set_caption('Sort Visualization') # 윈도우 타이틀

### 클래스 정의
- 오브젝트(막대기)
- 버블정렬
- 선택정렬
- 삽입정렬

In [38]:
# 막대기들
class Bars:
    def __init__(self, screen: pygame.display, bar_count: int, set_pos: list, set_size: list, margin: float, label: str=''):
        self.screen = screen
        self.margin = margin
        self.pos = set_pos
        self.size = set_size
        self.color = '#1C1C1C' # hex 사용
        self.label = font.render(label, True, '#333333')
        self.bars = []
        for i in range(bar_count):
            h = self.size[1] // bar_count * (i + 1)
            # bar (y, height)
            self.bars.append((self.pos[1] + self.size[1] - h, h, i))

    # 무작위 섞기
    # sample이 주어지면 주어진 대로 섞기
    def shuffle(self, sample: list = None):
        if sample == None:
            shuffle(self.bars)
        else:
            self.bars = [self.bars[i] for i in sample]

    # 막대기 고유 번호 가져오기
    def get(self, index):
        return self.bars[index][2]

    # 정렬되었는지 검사
    def is_sorted(self):
        for i in range(len(self.bars)):
            if self.bars[i][2] != 0: return False
        return True

    # 스크린에 그리기
    def draw(self):
        for i, bar in enumerate(self.bars):
            x = self.pos[0] + i * self.size[0] / len(self.bars)
            width = self.size[0] / len(self.bars)
            pygame.draw.rect(self.screen, self.color, [x, bar[0], width, bar[1]])
            self.screen.blit(self.label, (self.pos[0], self.pos[1] - 24))

class BubbleSort:
    def __init__(self, bars: Bars):
        self.bars = bars
        self.step = 0
        self.i = 0
        self.j = 0
    
    def switch(self, i, j):
        temp = self.bars.bars[i]
        self.bars.bars[i] = self.bars.bars[j]
        self.bars.bars[j] = temp

    def done(self):
        if self.i == len(self.bars.bars) - 1: return True
        else: return False
            

    def next(self):
        if not self.done():
            if self.bars.get(self.j) > self.bars.get(self.j + 1):
                self.switch(self.j, self.j + 1)
            
            self.j += 1
            if self.j == len(self.bars.bars) - self.i - 1:
                self.j = 0
                self.i += 1

        self.bars.draw()

class SelectionSort:
    def __init__(self, bars: Bars):
        self.bars = bars
        self.step = 0
        self.i = 0
        self.j = 1
        self.t = 0
    
    def switch(self, i, j):
        temp = self.bars.bars[i]
        self.bars.bars[i] = self.bars.bars[j]
        self.bars.bars[j] = temp

    def done(self):
        if self.i == len(self.bars.bars) - 1: return True
        else: return False
            

    def next(self):
        if not self.done():
            if self.bars.get(self.j) < self.bars.get(self.t):
                self.t = self.j
            
            self.j += 1
            if self.j == len(self.bars.bars):
                self.switch(self.i, self.t)
                self.i += 1
                self.t = self.i
                self.j = self.i + 1

        self.bars.draw()

class InsertionSort:
    def __init__(self, bars: Bars):
        self.bars = bars
        self.step = 0
        self.i = 1
        self.j = 1
    
    def switch(self, i, j):
        temp = self.bars.bars[i]
        self.bars.bars[i] = self.bars.bars[j]
        self.bars.bars[j] = temp

    def done(self):
        if self.i == len(self.bars.bars): return True
        else: return False
            
    def next(self):
        if not self.done():
            if self.bars.get(self.j - 1) > self.bars.get(self.j):
                self.switch(self.j - 1, self.j)
                self.j -= 1
            else:
                self.i += 1
                self.j = self.i
            
            if self.j == 0:
                self.i += 1
                self.j = self.i

        self.bars.draw()

### 루프

In [39]:
count = 40
sample = list(range(count))
shuffle(sample)

bars = [
    Bars(screen, count, [70, 80], [160, 120], 0, 'BubbleSort'),
    Bars(screen, count, [270, 80], [160, 120], 0, 'SelectionSort'),
    Bars(screen, count, [470, 80], [160, 120], 0, 'InsertionSort'),
]
Sorts = [ BubbleSort, SelectionSort, InsertionSort ]

sorts = []
for i in range(len(bars)):
    bars[i].shuffle(sample)
    sorts.append(Sorts[i](bars[i]))

done = False
while True:
    clock.tick(120)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            done = True
            break
    if done: break
    
    screen.fill('#FAFAFA')
    
    for s in sorts: s.next()

    pygame.display.flip()